In [1]:
# Database dependencies.
import pymongo
 
# Api dependencies
import json
import requests 

#Json dependencies
from pprint import pprint

#API key
from DriversConfig.config import API_KEY

#DataFrame dependencies
import pandas as pd 

#Webscrapping 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

#Random crap
import time

#Settings for headless mode.
options = webdriver.ChromeOptions()
options.add_argument('headless')

#path to the driver and load the options.
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Group Project2 - ProjectEnergy/Sebastian/DriversConfig/chromedriver",chrome_options = options)



In [2]:
# Database connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection --- creates database.
db = client.USEnergy

In [3]:
#Lower 48 list.
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Prices 

- Data is very structured, SQL can be a good idea here

In [18]:
#Define collection for prices
collection = db.Prices

#API key from config file.
api_key = API_KEY

#Empty dictionary to store prices
price_df = pd.DataFrame(columns = states)

#for state in states.
for state in states:

    #query each state. ALL = average combined rates, A = Annual. --> we can get more specific and get Residential, bussiness, etc. 
    priceQuery = f"ELEC.PRICE.{state}-ALL.A"

    #Api url.
    url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

    #Api reqeust. 
    statePriceRequest = requests.get(url)

    #Pulls list of list of with the information of prices and yaer.
    yearPricesList = statePriceRequest.json()['series'][0]['data']
    
    #Parse through the list and build DataFrame.
    for priceYear in yearPricesList:

        price_df.loc[priceYear[0],state] = priceYear[1]

#Converts the dataframe to a dictionary. 
post = price_df.to_dict()
collection.insert_one(post)




# Energy Generation

- Data is very unstructured, we might have to use NoSQL (Mongo_dB)


<img src="images/2019 US Energy Distribution.png">

In [19]:
#Define collection for Energy Generation.
collection = db.EnegryGeneration

#Dictionary to build.
energySources_dict = {}

#list of the US Energy Sources by EIA.
energySources = {"Natural Gas" : "NG",
                "Coal" : "COW",
                "Petroleum Liquids" : "PEL", 
                "Petroleum Coke" : "PC", 
                "Other Gasses": "OOG",
                "Nuclear" : "NUC",
                "Hydroelectric" : 'HYC',
                "Wind" : "WND",
                'Wood and Wood Derived Fuels' : 'WWW',
                'Other Biomass' : 'WAS',
                'Geothermal' : 'GEO',
                'All Other Renewables' : "AOR",
                "All Solar": 'TSN',
                'Other' : 'OTH',
                'Utility Scale Thermal' : 'STH',
                'Hydroelectic Pumped Storage' : 'HPS'
                }

#For state in states
for state in states:
    
    try:
        #For every statem loop each energy source.
        for energySourceKey, energySourceValue in zip(energySources.keys(), energySources.values()):
            
            #builds api query.
            priceQuery = f"ELEC.GEN.{energySourceValue}-{state}-99.A"
            
            #API url
            url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

            #Api reqeust. 
            energySourceRequest = requests.get(url, allow_redirects = False) 

            #Extracts the year-energy liss
            energySourceList = energySourceRequest.json()['series'][0]['data']

            #builds the state keys.
            if state not in energySources_dict.keys():
                energySources_dict[state] = {}

            #builds the energy source keys by year and energy production by source.
            if energySourceKey not in energySources_dict[state].keys():
                energySources_dict[state][energySourceKey] = {}
                energySources_dict[state][energySourceKey]['Year'] = []
                energySources_dict[state][energySourceKey]['K Megawatthours'] = []
                
            #builds the list for the years
            [energySources_dict[state][energySourceKey]['Year'].append(i[0]) for i in energySourceList]

            #Builds list for the values of each year.
            [energySources_dict[state][energySourceKey]['K Megawatthours'].append(i[1]) for i in energySourceList]
    
    #if energy surce is not found.
    except KeyError as K:
        print(f'{K}. {energySourceKey} not found in {state}.')

        time.sleep(5)

print('Done!')

post = energySources_dict

collection.insert_one(post)

'series'. Petroleum Coke not found in AL.
'series'. Petroleum Coke not found in AK.
'series'. Petroleum Coke not found in AZ.
'series'. Other Gasses not found in AR.
'series'. Petroleum Coke not found in CO.
'series'. Petroleum Coke not found in CT.
'series'. Coal not found in DC.
'series'. Nuclear not found in DE.
'series'. Geothermal not found in FL.
'series'. Other Gasses not found in GA.
'series'. Natural Gas not found in HI.
'series'. Petroleum Coke not found in ID.
'series'. Geothermal not found in IL.
'series'. Nuclear not found in IN.
'series'. Other Gasses not found in IA.
'series'. Other Gasses not found in KS.
'series'. Nuclear not found in KY.
'series'. Wind not found in LA.
'series'. Petroleum Coke not found in ME.
'series'. Petroleum Coke not found in MD.
'series'. Petroleum Coke not found in MA.
'series'. Geothermal not found in MI.
'series'. Geothermal not found in MN.
'series'. Petroleum Coke not found in MS.
'series'. Geothermal not found in MO.
'series'. Nuclear not 

# Energy Consumption by State

<img src="images/Energy Classification.png">


In [32]:
#Empty dataframe to store the data 
EndUseCosumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(4)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-electricity'] 

    for parseClass in parseClasses:

        yummySoup = soup.find_all("li", class_={parseClass})[0].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        EndUseCosumption_df.loc[typeEnergy,state] = marketShare

EndUseCosumption_df.to_pickle("Output Files/End-User_Consumption_by_Source.pkl")

EndUseCosumption_df

In [35]:
EndUseCosumption_df

,AL,AK,AZ,AR,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
Coal,(4.3%),(1.2%),(0.7%),(0.5%),(0.5%),(0.3%),(0.0%),(0.0%),(0.0%),(0.4%),...,(1.0%),(2.4%),(0.1%),(1.4%),(0.0%),(2.7%),(0.1%),(4.7%),(1.5%),(7.3%)
Naturalgas,(23.3%),(55.8%),(10.7%),(26.2%),(24.7%),(34.2%),(25.0%),(35.0%),(28.5%),(7.3%),...,(27.3%),(19.8%),(24.8%),(30.2%),(10.8%),(18.0%),(16.7%),(35.3%),(31.2%),(39.9%)
Petroleum,(38.9%),(37.8%),(58.0%),(43.0%),(57.7%),(46.1%),(55.3%),(21.6%),(51.8%),(59.4%),...,(38.1%),(49.4%),(61.7%),(49.9%),(60.0%),(49.4%),(55.2%),(37.6%),(40.9%),(38.2%)
Renewableenergy,(11.6%),(1.6%),(3.3%),(9.3%),(3.4%),(2.3%),(2.9%),(1.7%),(1.1%),(5.1%),...,(19.6%),(5.2%),(0.9%),(1.5%),(14.8%),(5.2%),(7.5%),(3.1%),(8.9%),(1.3%)
Electricity,(21.9%),(3.5%),(27.4%),(21.0%),(13.7%),(17.0%),(16.8%),(41.6%),(18.6%),(27.7%),...,(14.0%),(23.2%),(12.5%),(16.9%),(14.3%),(24.7%),(20.5%),(19.2%),(17.5%),(13.3%)


In [57]:
#Empty dataframe to store the data 
ElectricPowerConsumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(4)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-nuclear'] 

    for parseClass in parseClasses:

        #From the soup, the electricity generation is in the index 1.
        indexNum = 1

        #Nuclear energy is only for electricity generation. It only has 1 index = 0
        if parseClass == 'info-nuclear' :
            indexNum = 0

        yummySoup = soup.find_all("li", class_={parseClass})[indexNum].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        ElectricPowerConsumption_df.loc[typeEnergy,state] = marketShare

ElectricPowerConsumption_df.to_pickle("Output Files/Eletric_Power_Consumption_by_Source.pkl")

ElectricPowerConsumption_df


,AL,AK,AZ,AR,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
Coal,(24.7%),(18.7%),(30.5%),(48.3%),(0.0%),(52.3%),(1.2%),(0.0%),(9.6%),(15.6%),...,(21.0%),(27.1%),(29.2%),(71.7%),(0.0%),(12.4%),(5.6%),(94.1%),(54.2%),(89.8%)
Naturalgas,(34.6%),(45.6%),(27.8%),(25.1%),(39.4%),(25.8%),(40.8%),(0.0%),(83.9%),(64.0%),...,(8.4%),(13.0%),(41.9%),(17.1%),(0.0%),(43.8%),(7.9%),(1.7%),(20.9%),(0.4%)
Petroleum,(0.1%),(8.7%),(0.1%),(0.1%),(0.0%),(0.0%),(0.9%),(0.0%),(4.4%),(1.1%),...,(0.1%),(0.2%),(0.0%),(0.1%),(0.2%),(1.1%),(0.0%),(0.2%),(0.3%),(0.1%)
Renewableenergy,(8.4%),(26.9%),(11.1%),(5.2%),(48.7%),(21.8%),(5.5%),(0.0%),(2.1%),(4.3%),...,(70.6%),(12.1%),(18.2%),(11.1%),(99.8%),(6.9%),(76.9%),(4.0%),(7.8%),(9.7%)
Nuclearpower,(32.2%),(0.0%),(30.5%),(21.4%),(11.8%),(0.0%),(51.5%),(0.0%),(0.0%),(15.1%),...,(0.0%),(47.6%),(10.7%),(0.0%),(0.0%),(35.8%),(9.6%),(0.0%),(16.8%),(0.0%)
